<a href="https://colab.research.google.com/github/MaksPlank/Recommendation_sistem/blob/main/PRIJECT_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### PROJECT_4 : создание рекомендательной системы для онлайн-школы MasterMind.
    Стек инструментов : Python.Pandas, SQL.Metabase, PostrgeQSL.PgAgent, мат.статистика.А/B-Тест
    Задача прямая: создать рекомендательную систему для выявления интереса пользователей (каждому курсу будут рекомендованы ещё два)
    Задача косвенная: увеличение среднего чека покупки Курсов

Итогом работы станет файл, содержащий результаты всех промежуточных этапов: скрипты с комментариями, таблица рекомендаций и выводы.

Результат в следующем виде:
 - таблица: 1 курс/ 2 рекомендации
 - qsl-скрипты
 - pypton-код + комментарии
 - A/B-тест анализа результатов









ОПИСАНИЕ ДАННЫХ
- carts [таблица пользовательских корзин] sql-анализ
- art_items [таблица курсов, которые пользователи добавили в корзину] - sql-анализ

- data_sql.csv [таблица id пользователей и id курсов которые они приобрели]
     - user_id — ID пользователя
     - resource_id — ID курса

In [ ]:
import pandas as pd
from collections import Counter
from openpyxl import Workbook
import itertools

# 1. Выгрузка данных
# df, [34074 rows × 2 cols] - таблица  пользоввтелей и купленных ими курсов
df = pd.read_csv('data_sql.csv')


# 2. Подготовка словаря пар курсов, которые встречаются в опкупках у клиентов, сортировка
# Для каждого клиента в df собраны все купленные им курсы
# Зеркальные пары убираем сортировкой
user_groups = df.groupby('user_id')['resource_id'].apply(list).reset_index()
user_groups['sorted_resource'] = user_groups['resource_id'].apply(lambda x: sorted(x)) # display(user_groups)


# Узнаем сколько различных пар курсов встречаются вместе в покупках клиентов
# используем вспомогательный list(), itertools.combinations() и Counter()
courses = []
for i in user_groups['sorted_resource']:
    for j in itertools.combinations(i, 2):
        courses.append(j)  # display(courses)


# Количество уникальных пар в списке - 3989 уникальные пары
couplesCount = Counter(courses)
couplesUnique = list(couplesCount)  # display(len(c))


# Cловарь пар курсов:  3989 (key - пара, value - количество повторений)
couples_most_common = couplesCount.most_common()
couples_dict  = dict(couples_most_common)
couples_dictC = Counter(couples_dict) # display(couples_dictC)


# Таблица уникальных id курсов, за исключением курса, который ни разу не продавался
cources_and_purchase = df.groupby('resource_id')['user_id'].count().reset_index()
all_cources = cources_and_purchase['resource_id']


# Разделим на квантили количества повторений
# Пары с q=0.25 (повторений нет) и q=0.5 (3 повторения) рекоментовать не будем
# Для рекомендации оставим курсы с q=0.75 (9 повторений)
# Для курсов, не прошедших нимальгую границу: будем рекомендовать самую популярную пару кусов (551, 566): 797 повторений пары
# курс с id 551	 куплен 2935 раз,  566	- 2342 раза
# display(cources_and_purchase.sort_values(by='user_id', ascending = False))


# Запишем перечень пар и количество их повторений во вспомогательный df
df1 = pd.DataFrame({'couples': couples_most_common}, index=None)
df1['count_repeat'] = df1['couples'].apply(lambda x: x[1])     # количество
quantiles = df1['count_repeat'].quantile(q=[0.25, 0.5, 0.75])  # display('quantiles', quantiles)


# 3. Подготовка Таблицы рекомендаций:
# Функция рекомендаций на вход принимает номер курса, возврвщвет 2 курса, с max числом повторенй
# Цикл по всем уникальным курсам и запись результатов в датафрейм.
def Recommend(x):
    cource_list = []
    for i in couples_dict.keys():
        if (i[0] == x and couples_dictC[i] >= 9):          # условие: 1е знач. ключа соответствует номеру курса, повторений >=9
           cource_list.append((i[1], couples_dictC[i]))    # ключ и значение в список : курс-рекомндация
        if i[1] == x and couples_dictC[i] >= 9:            # условие: 2е знач.  ключа (рекомндация) соответствует номеру курса, повторений >=9
           cource_list.append((i[0], couples_dictC[i]))    # ключ и значение в список :
        if i[0] == x and couples_dictC[i] < 9:
           cource_list.append((551,0))                     # (551, 566): 797 - самая популярная пара кусов, рекомендуем их альтернативно
        if i[1] == x and couples_dictC[i] <9:
           cource_list.append((566,1))
    sorted_list = sorted(cource_list, key=lambda x:x[0], reverse = True) # сортировка от большего к меньшему числу повторений
    return sorted_list[:2]


# display(all_cources.apply(lambda x: Recommend(x))) # исходный вид выгрузки
# пустой df для таблицы
recommend_df = pd.DataFrame(columns = ['courses_ids', 'recommend_1', 'recommend_2'])


# Заполнение df рекомендаций
recommend_df['courses_ids'] = df['resource_id'].unique()
recommend_df['recommend_1']= all_cources.apply(lambda x: Recommend(x)[0][0])
recommend_df['recommend_2'] = all_cources.apply(lambda x: Recommend(x)[1][0])
display(recommend_df)


# Выгрузка таблицы рекомендаций в excel
recommend_df.to_excel('recommend_df.xlsx', sheet_name='table', index=False)

,courses_ids,recommend_1,recommend_2
0,356,1144,1125
1,515,1125,912
2,566,752,570
3,679,570,551
4,504,745,568
...,...,...,...
121,1182,1185,1141
122,865,566,566
123,866,566,566
124,1200,566,566
